In [1]:
# https://github.com/manmeet3591/kaggle_experiments/tree/main/pytorch

In [2]:
!pip uninstall torch -y
!pip uninstall torchvision -y
!pip install torch 
!pip install torchvision
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

Found existing installation: torch 1.7.0
Uninstalling torch-1.7.0:
  Successfully uninstalled torch-1.7.0
Found existing installation: torchvision 0.8.1
Uninstalling torchvision-0.8.1:
  Successfully uninstalled torchvision-0.8.1
     |████████████████████████████████| 831.4 MB 1.8 kB/s  eta 0:00:01     |█████▋                          | 145.6 MB 61.4 MB/s eta 0:00:12��█▊            | 512.3 MB 63.2 MB/s eta 0:00:06
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.2.7 requires torchvision<0.9,>=0.8, which is not installed.
easyocr 1.3.2 requires torchvision>=0.5, which is not installed.
allennlp 2.5.0 requires torchvision<0.10.0,>=0.8.1, which is not installed.
kornia 0.5.5 requires numpy<=1.19, but you have numpy 1.19.5 which is incompatible.
fastai 2.2.7 requires torch<1.8,>=1.7.0, but you have torch 1.9.0 which is incompatible.
allennlp 2.5.0 requires

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [6]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [8]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
            nn.BatchNorm2d(16),
            # https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
            nn.ReLU(),
            # https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html
            nn.MaxPool2d(kernel_size=2, stride=2))
            # https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [9]:
model = ConvNet(num_classes).to(device)

In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/5], Step [100/600], Loss: 0.1196
Epoch [1/5], Step [200/600], Loss: 0.0819
Epoch [1/5], Step [300/600], Loss: 0.0759
Epoch [1/5], Step [400/600], Loss: 0.0340
Epoch [1/5], Step [500/600], Loss: 0.0314
Epoch [1/5], Step [600/600], Loss: 0.0456
Epoch [2/5], Step [100/600], Loss: 0.0127
Epoch [2/5], Step [200/600], Loss: 0.0712
Epoch [2/5], Step [300/600], Loss: 0.0228
Epoch [2/5], Step [400/600], Loss: 0.0739
Epoch [2/5], Step [500/600], Loss: 0.0466
Epoch [2/5], Step [600/600], Loss: 0.0196
Epoch [3/5], Step [100/600], Loss: 0.0121
Epoch [3/5], Step [200/600], Loss: 0.0255
Epoch [3/5], Step [300/600], Loss: 0.0441
Epoch [3/5], Step [400/600], Loss: 0.0237
Epoch [3/5], Step [500/600], Loss: 0.0625
Epoch [3/5], Step [600/600], Loss: 0.0100
Epoch [4/5], Step [100/600], Loss: 0.0026
Epoch [4/5], Step [200/600], Loss: 0.0019
Epoch [4/5], Step [300/600], Loss: 0.0587
Epoch [4/5], Step [400/600], Loss: 0.0677
Epoch [4/5], Step [500/600], Loss: 0.0296
Epoch [4/5], Step [600/600], Loss:

In [12]:
# Test the model
# https://discuss.pytorch.org/t/model-eval-vs-with-torch-no-grad/19615
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [18]:
with torch.no_grad():
    correct = 0
    total = 0
    print(len(test_loader))
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        #print(labels)
        outputs = model(images)
        #print(torch.max(outputs.data))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

100
Test Accuracy of the model on the 10000 test images: 98.97 %


In [19]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')